# POC DataWarehouse Cloud
## Author:Rodrigo Sobarzo

In [161]:
#Global Libraries
import pandas as pd
import random
from random import seed
from random import gauss
import gc
import os, psutil, numpy as np
import datetime
from datetime import time, timedelta


#Global variables
outputdir = 'Data\\'
sep = ';'
var_boolean = ['Y', 'N']

#Clientes
cus_records = 1000000
cus_table = 'CUSTOMER'
cus_fields = ['CUS_ID','CUS_NAME','CUS_LASTNAME','CUS_REGION','CUS_CITY','CUS_GEOLAT','CUS_GEOLON','CUS_ADDRESS'
              ,'CUS_PHONE','CUS_EMAIL','CUS_AGE','CUS_GEN']
cus_values =['','','','','',0.0000,0.0000,'','','','','']
emails = ['@gmail.com', '@outlook.com', '@yahoo.com', '@aol.com' ]

#proveedores
ven_records = 10000
#Productos
pro_records = 200000
pro_table = 'PRODUCT'
pro_fields = ['PRO_ID','PRO_NAME','PRO_EAN','VEN_ID','PRO_IMPORTED','PRO_LENGTH','PRO_WIDTH','PRO_HEIGTH'
              ,'PRO_UNITOFMESURE','PRO_PRIVATEBRAND','PRO_PRICE','PRO_COST','PRO_TAX1RATE','PRO_TAX2RATE', 'CAT_ID']
pro_values = ['','','','','','','','','','','','','','','']
pro_ean = [5800000000000, 8900000000000]
pro_lenght = [10, 200]
pro_width = [10, 200]
pro_height = [10, 200]
pro_UOM = ['UND','KG','CAJ','DSP']
pro_cost = [100, 100000]
pro_margin = [10 , 50]
pro_tax1rate = '0,19'

#Locales
sto_records = 500
sto_table = 'STORE'
sto_fields = ['STO_ID','ORG_ID','STO_NAME','CIT_ID','STO_FLOOR','STO_OPENDATE','STO_CLOSEDATE','STO_SSS','STO_POSQTY']
sto_values = ['','','','','','','','','']
org_id = ['Brand1', 'Brand2', 'Brand3', 'Brand4']
sto_floor = [1000,1500,3000,4000]
sto_posqty = [4,5,8,12]
tmp_sto_startDate = datetime.datetime.strptime('2010-01-01', '%Y-%m-%d')
tmp_sto_endDate = datetime.datetime.strptime('2019-12-31', '%Y-%m-%d')
sto_openDate = [tmp_sto_startDate , tmp_sto_endDate]
#print(tables)

#Transacciones de venta - Cabecera
sah_records = 100000
sah_table = 'SALE_HEADER'
sah_fields = ['SAH_ID','CUS_ID','SAH_TIMESTAMP','STO_ID','SAH_POSNUM','SAH_TRXNUM','SAH_TOTPRODQTY','SAH_DOCTYPE'
              ,'SAH_PAYMENTTYPE','SAH_TOTPRODAMT','SAH_TOTTAX1AMT','SAH_TOTTAX2AMT']
sah_values = ['','','','','','','','','','','','']
tmp_sah_StartDate = datetime.datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
tmp_sah_endDate = datetime.datetime.strptime('2021-01-31 23:59:59', '%Y-%m-%d %H:%M:%S')
sah_trxDateRange = [tmp_sah_StartDate , tmp_sah_endDate]
tmp_sah_OpenTime = datetime.time(8, 30, 0)
tmp_sah_CloseTime = datetime.time(21, 30, 0)
sah_trxTimeRange = [tmp_sah_OpenTime, tmp_sah_CloseTime]
sah_docType = ['1', '2'] #1=BOLETA 2=FACTURA
sah_paymentType = ['1', '2', '3'] #1=Efectivo 2=Debito 3=Credito

#Transacciones de venta - Detalle
sad_ProQtyRange = [1,40]
sad_ProUnitRange = [1,10]
sad_table = 'SALE_DETAIL'
sad_fields = ['SAH_ID','SAH_TIMESTAMP','STO_ID','SAH_POSNUM','SAH_TRXNUM','PRO_ID','SAD_PRODQTY','SAD_PRODAMT'
              ,'SAD_TAX1AMT','SAD_TAX2AMT']
sad_values = ['','','','','','','','','','']

In [74]:
def getRandomRecord(df):
    MaxRow= len(df)-1
    randValue = random.randrange(0, MaxRow, 1)
    #print(df.iloc[randValue])
    return(df.iloc[randValue])

def usage():
    process = psutil.Process(os.getpid())
    return process.memory_info()[0] / float(2 ** 20)

#def getRandomDate(startDate, endDate) :
#def getRandomDate(startDate: datetime, endDate: datetime) -> datetime:
#    timeBetweenDates = endDate - startDate
#    daysBetweenDates = timeBetweenDates.days
#   #print(timeBetweenDates, daysBetweenDates)
#    randomNumberOfDays = random.randrange(daysBetweenDates)
#    randomDate = startDate + datetime.timedelta(days=randomNumberOfDays)
#    return(randomDate)

#def getRandomDate(startDate: datetime, endDate: datetime) -> datetime:
#     return datetime.datetime.fromtimestamp(random.randrange(
#        round(startDate.timestamp()), round(endDate.timestamp())
#    ))

def getRandomDate(startDate, endDate):
    delta = endDate - startDate
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return startDate + timedelta(seconds=random_second)

#def getRandomTime(start: time, end: time) -> time:
#    print(start, end)
#    tmpStartDate = (datetime.datetime.combine(dt0 := datetime.datetime.fromtimestamp(0), start))
#    tmpEndDate = (datetime.datetime.combine(dt0 if start < end else dt0 + timedelta(days=1), end))
#    return getRandomDate(tmpStartDate,tmpEndDate).time()


In [27]:
#generar archivo de datos clientes
#cargar los dataframes
dfCities = pd.read_csv('InputData\\cities.csv', sep=";" , encoding='utf_8')
dfNames = pd.read_csv('InputData\\names.csv', sep=";" , encoding='utf_8')
dfSurnames = pd.read_csv('InputData\\surnames.csv', sep=";" , encoding='utf_8')

#start timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )

#generar la cabecera del archivo de salida
oFileName= ('%s%s.csv' % (outputdir, cus_table))
oFile=open(oFileName , "w" , encoding='utf-8')
oFile.write(';'.join(cus_fields)+'\n')

#generar los registros del archivo de salida
for i in range(1,cus_records+1):
        df1 = getRandomRecord(dfCities)
        df2 = getRandomRecord(dfNames)
        df3 = getRandomRecord(dfSurnames)
        cus_values[0]= str(i) 
        cus_values[1]= df2.NAME
        cus_values[2]= df3.surname
        cus_values[3]= df1.region
        cus_values[4]= df1.city
        cus_values[5]= "{:.6f}".format(float(df1.lat.replace(',','.')) + gauss(0, 0.009))
        cus_values[6]= "{:.6f}".format(float(df1.lng.replace(',','.')) + gauss(0, 0.009))
        cus_values[7]= 'CUS_ADDRESS' + str(i)
        cus_values[8]= str(56910000000 + i)
        #print(cus_values)
        cus_values[9]= cus_values[1].replace(" ", "")+'.'+cus_values[2].replace(" ", "")+ emails[random.randrange(0, len(emails), 1)] 
        cus_values[10]= str(random.randrange(18, 90, 1))
        cus_values[11]= df2.GEN      
        oFile.write(';'.join(cus_values)+'\n')
#cerrar el archivo
#print(usage())
oFile.close()

#Liberar df
del [[dfCities,dfNames,dfSurnames,df1, df2, df3]]
gc.collect()
dfCities=pd.DataFrame()
dfNames=pd.DataFrame()
dfSurnames=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
#print(usage())

#End timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )

TimeStamp: 2022-04-24 22:51:41.110807
TimeStamp: 2022-04-24 22:52:25.092032


In [95]:
# generar archivo de productos
#cargar los dataframes
dfCat = pd.read_csv('InputData\\categories.csv', sep=";" , encoding='utf_8')
dfVen = pd.read_csv('InputData\\vendor.csv', sep=";" , encoding='utf_8')

#start timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )

#generar la cabecera del archivo de salida
oFileName= ('%s%s.csv' % (outputdir, pro_table))
oFile=open(oFileName , "w", encoding='utf-8')
oFile.write(';'.join(pro_fields)+'\n')

#generar los registros del archivo de salida
for i in range(1,pro_records+1):
        df1 = getRandomRecord(dfCat)
        df2 = getRandomRecord(dfVen) 
        pro_values[0]= str(i) 
        pro_values[1]= 'PRO_NAME' + str(i) 
        pro_values[2]= str(random.randrange(pro_ean[0], pro_ean[1], 1))
        pro_values[3]= str(df2.ven_id)
        pro_values[4]= random.choice(var_boolean)
        pro_values[5]= str(random.randrange(pro_lenght[0], pro_lenght[1], 10))
        pro_values[6]= str(random.randrange(pro_width[0], pro_width[1], 10))
        pro_values[7]= str(random.randrange(pro_height[0], pro_height[1], 10))
        pro_values[8]= random.choice(pro_UOM)
        pro_values[9]= random.choice(var_boolean)
        tmp_cost = (random.randrange(pro_cost[0], pro_cost[1], 1))
        tmp_price = tmp_cost *(1+ (random.randrange(pro_margin[0], pro_margin[1], 1))/100)
        pro_values[10]= str(int(tmp_price))
        pro_values[11]= str(int(tmp_cost))
        pro_values[12]= pro_tax1rate 
        pro_values[13]= df1.CAT_REF_TAX2   
        pro_values[14]= str(df1.CAT_LVL1_ID)
        #print(pro_values)
        oFile.write(';'.join(pro_values)+'\n')
#cerrar el archivo
oFile.close()

#Liberar df
del [[dfCat,dfVen,df1, df2]]
gc.collect()
dfCat=pd.DataFrame()
dfVen=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
#print(usage())

#End timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )

TimeStamp: 2022-04-25 01:14:06.231385
TimeStamp: 2022-04-25 01:14:36.265871


In [29]:
# generar archivo de Tiendas
#cargar los dataframes
dfCities = pd.read_csv('InputData\\cities.csv', sep=";" , encoding='utf_8')

#start timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )
#generar la cabecera del archivo de salida
oFileName= ('%s%s.csv' % (outputdir, sto_table))
oFile=open(oFileName , "w", encoding='utf-8')
oFile.write(';'.join(sto_fields)+'\n')

#generar los registros del archivo de salida
for i in range(1,sto_records+1):
        df1 = getRandomRecord(dfCities)
        sto_values[0]= str(i) 
        sto_values[1]= random.choice(org_id)
        sto_values[2]= 'STO_' + str(i) + '_'+df1.city
        sto_values[3]= str(df1.id)
        varFloorPosQty = random.randrange(0, len(sto_floor), 1)
        sto_values[4]= str(sto_floor[varFloorPosQty])
        var_Opendate = getRandomDate(sto_openDate[0] , sto_openDate[1]) 
        sto_values[5]= var_Opendate.strftime("%Y-%m-%d")
        sto_values[6]= ''
        sto_values[7]=  'Y' if (datetime.datetime.today() - var_Opendate).days > 365 else 'N'
        sto_values[8]= str(sto_posqty[varFloorPosQty])
        #print(sto_values)
        oFile.write(';'.join(sto_values)+'\n')
#cerrar el archivo
oFile.close()

#Liberar df
del [[dfCities,df1]]
gc.collect()
dfCities=pd.DataFrame()
df1=pd.DataFrame()
#print(usage())

#End timestamp
print('TimeStamp: %s' % ( datetime.datetime.now()) )

TimeStamp: 2022-04-24 22:53:20.717045
TimeStamp: 2022-04-24 22:53:20.931485


In [162]:
# generar archivo de Transacciones de venta
#cargar los dataframes
#previamente deben estar cargados los archivos STORE CUSTOMER y PRODUCT.
dfStore = pd.read_csv('Data\\STORE.csv', sep=";" , encoding='utf_8' )
dfProduct = pd.read_csv('Data\\PRODUCT.csv', sep=";" , encoding='utf_8')
dfCustomer = pd.read_csv('Data\\CUSTOMER.csv', sep=";" , encoding='utf_8')
#start timestamp
starttime = datetime.datetime.now()
print('TimeStamp: %s' % (starttime) )

#generar la cabecera del archivo de salida SALES_HEADER
oFileName= ('%s%s.csv' % (outputdir, sah_table))
oFileSah=open(oFileName , "w")
oFileSah.write(';'.join(sah_fields)+'\n')

#generar la cabecera del archivo de salida SALES_DETAIL
oFileName= ('%s%s.csv' % (outputdir, sad_table))
oFileSad=open(oFileName , "w")
oFileSad.write(';'.join(sad_fields)+'\n')

#generar los registros del archivo de salida cabecera
for i in range(1,sah_records+1):
        df1 = getRandomRecord(dfStore)
        df2 = getRandomRecord(dfCustomer)
        sah_values[0]= str(i) 
        sah_values[1]= str(df2.CUS_ID)
        tmp_sahDate = getRandomDate(sah_trxDateRange[0] , sah_trxDateRange[1]).date() 
        tmp_sahStartTime= datetime.datetime.combine(tmp_sahDate, sah_trxTimeRange[0])
        tmp_sahEndTime= datetime.datetime.combine(tmp_sahDate, sah_trxTimeRange[1])
        tmp_sahTimeStamp = getRandomDate (tmp_sahStartTime,tmp_sahEndTime)
        sah_values[2]= tmp_sahTimeStamp.strftime("%Y-%m-%d %H:%M:%S")
        sah_values[3]= str(df1.STO_ID)
        sah_values[4]= str(random.randrange(1,df1.STO_POSQTY, 1))
        sah_values[5]= str(i) 
        sah_values[6]= '0' #se completa con el detalle de la transaccion
        sah_values[7]= random.choice(sah_docType)
        sah_values[8]= random.choice(sah_paymentType)
        sah_values[9]= '0' #se completa con el detalle de la transaccion
        sah_values[10]= '0' #se completa con el detalle de la transaccion
        sah_values[11]= '0' #se completa con el detalle de la transaccion
        tmp_sadvalues_0 = sah_values[0]
        tmp_sadvalues_1 = sah_values[2]
        tmp_sadvalues_2 = sah_values[3]
        tmp_sadvalues_3 = sah_values[4]
        tmp_sadvalues_4 = sah_values[5]
        
        #print(sah_values)
        tmp_sah_totprodqty=random.randrange(sad_ProQtyRange[0], sad_ProQtyRange[1], 1)
        sah_values[6]= str(tmp_sah_totprodqty)
        tmp_sah_totprodamt=0
        tmp_sah_tottax1amt =0
        tmp_sah_tottax2amt =0
        #print(tmp_sah_totprodqty)
        for j in range(1, tmp_sah_totprodqty+1):
            #print(i, j)
            df3 = getRandomRecord(dfProduct)
#            print(sad_values)
            sad_values[0]=str(tmp_sadvalues_0)
            sad_values[1]=str(tmp_sadvalues_1)
            sad_values[2]=str(tmp_sadvalues_2)
            sad_values[3]=str(tmp_sadvalues_3)
            sad_values[4]=str(tmp_sadvalues_4)
            sad_values[5]=str(df3.PRO_ID)
            tmp_sadProdQty = random.randrange(sad_ProUnitRange[0], sad_ProUnitRange[1],1)
            sad_values[6]=str(tmp_sadProdQty)
            #print(df3)
            tmp_sadProdAmp = tmp_sadProdQty*df3.PRO_PRICE
            sad_values[7]=str(tmp_sadProdAmp)
            sad_values[8]=str(round(tmp_sadProdAmp*float(df3.PRO_TAX1RATE.replace(",", "."))))
            sad_values[9]=str(round(0+tmp_sadProdAmp*float(df3.PRO_TAX2RATE.replace(",", "."))))
            tmp_sah_totprodamt = tmp_sah_totprodamt + int(tmp_sadProdAmp)
            tmp_sah_tottax1amt = tmp_sah_tottax1amt + int(sad_values[8])
            tmp_sah_tottax2amt = tmp_sah_tottax2amt + int(sad_values[9])
            #print(sad_values)
            oFileSad.write(';'.join(sad_values)+'\n')
            
        sah_values[9]= str(tmp_sah_totprodamt)
        sah_values[10]= str(tmp_sah_tottax1amt)
        sah_values[11]= str(tmp_sah_tottax2amt)
        #print(sah_values)
        oFileSah.write(';'.join(sah_values)+'\n')
#cerrar el archivo
oFileSah.close()
oFileSad.close()
#Liberar df
del [[dfStore,dfProduct,dfCustomer, df1, df2 ,df3]]
gc.collect()
dfStore=pd.DataFrame()
dfProduct=pd.DataFrame()
dfCustomer=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
#print(usage())

#End timestamp
endtime = datetime.datetime.now()  
print('TimeStamp: %s' % (endtime) )
print(endtime - starttime)

TimeStamp: 2022-04-26 07:06:52.403185
TimeStamp: 2022-04-26 07:14:35.124845
0:07:42.721660


In [ ]:
import getpass
pwd = getpass.getpass("Enter password")

In [ ]:
import snowflake.connector
conn = snowflake.connector.connect(user='rsobarzo',password=str(pwd),account='pq40291.us-east-2.aws')


In [ ]:
#realizar una query de prueba
str_query="Select * from POC_DB.PUBLIC.CUSTOMER LIMIT 10000;"


In [ ]:
#retornar el resultado de la query
res = conn.cursor().execute(str_query).fetchall()

In [ ]:
print(len(res))

print(res[0])


In [ ]:
#truncar tabla del modelo
str_truncate = 'truncate table POC_DB.PUBLIC.CUSTOMER'
res = conn.cursor().execute(str_truncate).fetchall()
print(len(res))

print(res[0])

In [ ]:
#generar datos para la tabla

for x in range(len(tables)):
 for i in range(0,records):
    Col0=i
    Col1='Customer_name'
    Col2='Customer_lastname'
    Col3='Customer_address'
    Col4= 56910000000
    Col5='Customer_email'
    Col1=Col1 + str(i)
    Col2=Col2 + str(i)
    Col3=Col3 + str(i)
    Col4=Col4 + i
    Col5=Col5 + str(i)
    #print(
    
    conn.cursor().execute(
        "INSERT INTO POC_DB.PUBLIC.CUSTOMER "
                        "VALUES(%s, %s, %s, %s, %s, %s )", (Col0, Col1, Col2, Col3, Col4, Col5))




In [ ]:
for i=1 to records
con.cursor().execute(
    "INSERT INTO testtable(col1, col2) "
    "VALUES(%s, %s)", (
        789,
        'test string3'
    ))

In [ ]:
#revisar estructura de la tabla 
import pandas
df = pandas.read_sql(str_query, conn)
df.info()
